# duckdb


Konventering til fgb

In [18]:
import geopandas as gpd
import duckdb
import os
import fiona

In [35]:
def convert_to_flatgeobuf(input_file, output_file, layer=None):
    file_ext = os.path.splitext(input_file)[1].lower()

    # gml is good to go
    if file_ext == '.gml':
        if layer is None:
            raise ValueError("For GML-filer må 'layer' spesifiseres (f.eks. 'ArealressursFlate').")
        gdf = gpd.read_file(input_file, layer=layer)
    # gdb is good to go
    elif file_ext == '.gdb':
        if layer is None:
            layers = fiona.listlayers(input_file)
            print("Tilgjengelige lag i GDB:", layers)
            layer = layers[0]
        gdf = gpd.read_file(input_file, layer=layer)

    # Får ikke til sosi -_-
    elif file_ext == '.sos':
        # Bruk absolutt bane for å være sikker
        abs_path = os.path.abspath(input_file)
        # Først prøv med prefikset "SOSI:" og engine="fiona"
        try:
            print("Prøver å lese SOSI-fil med 'SOSI:' prefiks og engine='fiona'")
            gdf = gpd.read_file("SOSI:" + abs_path, engine="fiona")
        except Exception as e:
            print("Feil med 'SOSI:' prefiks og engine='fiona':", e)
            # Deretter prøv uten prefiks
            try:
                print("Prøver å lese SOSI-fil uten prefiks med engine='fiona'")
                gdf = gpd.read_file(abs_path, engine="fiona")
            except Exception as e2:
                print("Feil med engine='fiona' uten prefiks:", e2)
                raise e2
    else:
        raise ValueError(f"Ukjent eller ikke støttet filtype: {file_ext}")

    # Filtrer ut kun polygoner (Polygon og MultiPolygon)
    gdf = gdf[gdf.geom_type.isin(['Polygon', 'MultiPolygon'])]
    print("Geometrityper etter filtrering:")
    print(gdf.geom_type.value_counts())

    # Eksporter til FlatGeobuf
    gdf.to_file(output_file, driver="FlatGeobuf")
    print(f"Lagret polygoner som {output_file}")

    return gdf

input_gml = "data/42_25832_ar50_gml.gml"
input_gdb = "data/42_25832_ar50_gdb.gdb"
input_sosi = "data/42_25832_ar50_sosi.sos"
output_fgb = "data/AR50_flater.fgb"
# gdb filen så må man skrive ar50 på layer
# gml filen må man skrive ArealressursFlate på layer
convert_to_flatgeobuf(input_gml, output_fgb, layer="ArealressursFlate")

Geometrityper etter filtrering:
Polygon    179790
Name: count, dtype: int64
Lagret polygoner som data/AR50_flater.fgb


,gml_id,lokalId,navnerom,informasjon,områdeId,originalDatavert,kopidato,målemetode,oppdateringsdato,arealtype,skogbonitet,treslag,jordbruk,vegetasjonsdekke,kartstandard,geometry
0,idfbb8c036-19e3-4a9a-b2f6-0c2d35695c1d,176214,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4212,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,30,13,31,98,98,AR50,"POLYGON ((485218.613 6517125.999, 485201.686 6..."
1,id36b5f040-baf2-4727-b0a1-cc90cb4bd608,176213,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4212,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,30,11,99,98,98,AR50,"POLYGON ((485586.131 6519504.745, 485607.835 6..."
2,ide33f3b18-38c2-4514-ae13-ca2df0a2e41a,176212,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4212,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,60,12,31,98,98,AR50,"POLYGON ((487006.634 6519259.077, 486992.389 6..."
3,idd556feed-1ea8-48c4-af32-9ad8e3da9fe7,176211,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4212,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,60,11,39,98,98,AR50,"POLYGON ((483970.068 6519640.093, 483990.43 65..."
4,id5f39e277-26d6-42b0-bacf-f81ac3a628d2,176210,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4212,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,30,11,31,98,98,AR50,"POLYGON ((485413.573 6520531.763, 485458.321 6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179785,id8d863682-6cb3-4c6b-9190-8eefa6cf6633,5,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4206,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,60,11,39,98,98,AR50,"POLYGON ((358617.147 6442580.922, 358631.381 6..."
179786,idff57878f-0fbb-4ee5-9fa5-d8e364e9ff58,4,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4206,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,30,13,31,98,98,AR50,"POLYGON ((358002.134 6441284.427, 357998.139 6..."
179787,idbbd694b0-068f-4308-ab6e-8c5d3f34e011,3,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4206,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,50,98,39,98,55,AR50,"POLYGON ((357579.6 6444891.118, 357578.268 644..."
179788,idce0ff6b8-8fa6-4cb7-8476-08e2415bb4ad,2,NO_NIBIO_AR50_2022_01,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,4206,NIBIO,2024-11-17T00:00:00,64,2022-01-18T00:00:00,20,98,98,24,98,AR50,"POLYGON ((357225.405 6444709.267, 357232.189 6..."


In [16]:
# Koble til DuckDB
conn = duckdb.connect(database=":memory:")

# Installer og last spatial-utvidelsen
conn.execute("INSTALL spatial;")
conn.execute("LOAD spatial;")

# Sjekk at spatial-utvidelsen er lastet
print(conn.execute("SELECT * FROM duckdb_extensions() WHERE extension_name = 'spatial'").df())

# Sjekk at FlatGeobuf-filen eksisterer
fgb_path = 'Data/AR50_flater.fgb'  # Erstatt med riktig filbane
print(f"Eksisterer filen? {os.path.exists(fgb_path)}")

# Les FlatGeobuf-filen og opprett en tabell
conn.execute(f"""
CREATE TABLE ar50 AS
SELECT * FROM ST_Read('{fgb_path}')
""")

# Bekreft at tabellen er opprettet
print(conn.execute("SELECT * FROM ar50 LIMIT 5").df())

  extension_name  loaded  installed  \
0        spatial    True       True   

                                        install_path  \
0  /Users/ineantonsen/.duckdb/extensions/v1.2.1/o...   

                                         description aliases  \
0  Geospatial extension that adds support for wor...      []   

  extension_version install_mode installed_from  
0           2905968   REPOSITORY           core  
Eksisterer filen? True
                                   gml_id  lokalId               navnerom  \
0  id169b1688-10c6-4c91-98a0-699076088a4c    97254  NO_NIBIO_AR50_2022_01   
1  id743ec309-3b59-4cad-bde6-ed6ea386cf62   137183  NO_NIBIO_AR50_2022_01   
2  idc68f2cfa-11a4-4a75-b16f-9e433ebfafbd    50982  NO_NIBIO_AR50_2022_01   
3  id4d85f2e2-a47a-4c97-856e-67238e6131e2    50981  NO_NIBIO_AR50_2022_01   
4  id563d71e8-898f-4905-834e-1b52e2e3244f   135365  NO_NIBIO_AR50_2022_01   

                                         informasjon  områdeId  \
0  AR50 fra AR5 årsversjon 

In [21]:
input_gdb = "data/42_25832_ar50_gdb.gdb"
layers = fiona.listlayers(input_gdb)
print("Tilgjengelige lag i GDB:", layers)

Tilgjengelige lag i GDB: ['ar50']


In [30]:
input_sosi = os.path.abspath("Data/42_25832_ar50_sosi.sos")
print("Absolutt bane:", input_sosi)

Absolutt bane: /Users/ineantonsen/Desktop/Bachelor-V25/kaidata_geolake/Ine/data/42_25832_ar50_sosi.sos
